# Problem Set 5

This problem set offers an opportunity to apply the technology in Rust(1987), Aguirregabiria and Mira(2002), and Hotz and Miller (1993).  

In [1]:
import pandas as pd
import os
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d, interp2d
from scipy.optimize import minimize_scalar,minimize
import matplotlib.pyplot as plt
%matplotlib inline
from quantecon.markov import DiscreteDP
from scipy.stats import norm
from math import sqrt
import matplotlib.pyplot as plt
from scipy import optimize
from scipy.optimize import brentq
import time
import statsmodels.api as sm

In [238]:
df = pd.read_csv('Pset5DataQ1(1).csv')
df.head()

,id,time,x,d
0,1,1,4,0
1,2,1,8,1
2,3,1,5,0
3,4,1,7,1
4,5,1,3,0


In [239]:
data2 = df[(df.id==1)&(df.time==1)]
if data2.d.any()==1:
    print('OK')
else:
    print('not ok')

not ok


In [240]:
# set paramters
β,M,γ,N,T = 0.95,9,0.5772,500,20

In [241]:
iteration = 1000
v = np.zeros((2,M+1,iteration))
x_t = np.array([i for i in range(M+2)])
x_t[-1] = x_t[-2]
tolerance = 1e-5
x_t

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9])

In [242]:
def value(θ):
    θ1,θ2 = θ[0],θ[1]
    for i in range(1,iteration):
        for x in x_t[:-1]:
            EV = np.log(np.exp(v[0,(x<9)*(x+1)+(x==9)*x,i-1]) + np.exp(v[1,0,i-1])) + γ
            v[0,x,i] = -θ1*x + β*EV
            v[1,x,i] = -θ1*x - θ2 + β*EV
            
    
        diff = v[:,:,i] - v[:,:,i-1]
        if np.average(np.abs(diff)) < tolerance:
            break
    return v[:,:,i] 

In [243]:

def l_q1(θ,f,x,d):
    v0 = f(θ)[0,:]
    v1 = f(θ)[1,:]
    l = ((d==1)*np.exp(v1[x])+(d==0)*np.exp(v0[x]))/(np.exp(v0[x])+np.exp(v1[x]))
    return l

In [244]:
def L(θ):
    θ1,θ2 = θ[0], θ[1]
    data = df
    L = 0
    log_l = np.log(l_q1(θ,value,df.x,df.d))                    
    return -np.sum(log_l)

In [245]:
θ0 = np.array([0.1,5])
bnd = ((0.0001,1),(1,10))
result_a = minimize(L,θ0,bounds=bnd)

In [226]:
result_a.x

array([0.44578392, 3.61141111])

In [275]:
hess_a = np.linalg.inv(result_a.hess_inv([[1,0],[0,1]]))
se_a = (np.diag(hess_a)/len(df))**(0.5)

Standard errors are following:

In [276]:
se_a

array([0.01000005, 0.39951013])

## Question 1(b)

In [248]:
p_1 = np.zeros(len(x_t[:-1]))
p_0 = np.zeros(len(x_t[:-1]))
for x in x_t[:-1]:
    p_1[x] =  np.sum(df.d[(df.d==1)&(df.x==x)])/len(df.d[df.x==x])
    p_0[x] = 1 - p_1[x]


In [249]:
Δv = np.log(p_1/p_0)
Δv

array([-3.8376148 , -2.96033035, -1.90987021, -1.39967249, -0.65778996,
       -0.22086824,  0.13071211,  0.50134688,  1.22050211,  0.81831032])

In [250]:
def Δv_hat(θ,x):
    θ1, θ2 = θ[0],θ[1]
    delta_v = -θ2 + β*(np.log(1+p_1[0]/p_0[0]) + np.log(1+np.exp(Δv[(x<9)*(x+1)+(x==9)*x]))) + β*θ1*(x+(x<M))
    return delta_v

In [251]:
def l(θ,x,d):
    p_ratio = np.exp(Δv_hat(θ,x))
    p_1 = p_ratio/(1+p_ratio)
    p_0 = 1 - p_1
    return (d==1)*p_1 + (d==0)*p_0

In [252]:
def L_2(θ):
    log_l = np.log(l(θ,df.x,df.d))
    return -np.sum(log_l)

In [253]:
θ0 = np.array([1,1])
bnd = ((0.00001,10),(1,10))
result_b = minimize(L_2,θ0,bounds=bnd)

In [254]:
result_b.x

array([0.44578392, 3.61141111])

In [273]:
hess_b = np.linalg.inv(result_b.hess_inv([[1,0],[0,1]]))
se_b = (np.diag(hess_b)/len(df))**(0.5)

Standard errors are following:

In [274]:
se_b

array([1.06941853, 0.27374948])

## Question 1(c)

In [257]:
# The idea is to iterate in order to find probability 
def P_0(θ):
    iteration = 2000
    toler = 1e-3
    θ1, θ2 = θ[0],θ[1]
    diff_v = np.zeros((len(x_t)-1,iteration+1))
    p_0 = np.zeros(len(x_t)-1) + 0.5
    p_1 = 1 - p_0
    for m in range(iteration):
        diff_v[:,m+1] = -θ2 + β*(np.log(1+p_1[0]/p_0[0]) + np.log(1+np.exp(diff_v[(x_t[:-1]<9)*(x_t[:-1]+1)+(x_t[:-1]==9)*x_t[:-1],m]))) + β*θ1*(x_t[:-1]+(x_t[:-1]<M))
        p_0_new = 1/(1+np.exp(diff_v[:,m+1]))
        p_1_new = 1 - p_0_new
        if np.sum(np.abs(p_0_new-p_0))<toler:
            break
        else:
            p_0, p_1 = p_0_new, p_1_new
    return p_0

In [258]:
θ = np.array([0.3,6])
P_0(θ)

array([0.99669002, 0.99559697, 0.99414216, 0.99220509, 0.98962441,
       0.98618323, 0.98158841, 0.97546706, 0.96763883, 0.96763883])

In [259]:
def l_q3(θ,x,d):
    p_0 = P_0(θ)
    p_1 = 1 - p_0
    return (d==1)*p_1[x] + (d==0)*p_0[x]

In [260]:
def L_3(θ):
    log_l = np.log(l_q3(θ,df.x,df.d))
    return -np.sum(log_l)

In [261]:
θ0 = np.array([0.3,5])
bnd = ((0.00001,0.53),(1,10))
result_3 = minimize(L_3,θ0,bounds=bnd)

Estimations are:

In [262]:
result_3.x

array([0.36269375, 3.31551376])

In [271]:
hess_3 = np.linalg.inv(result_3.hess_inv([[1,0],[0,1]]))
se_3 = (np.diag(hess_3)/len(df))**0.5

In [272]:
se_3

array([4.27049492, 0.51860556])

## Question 2

Use the following transformation for the derivative:

\begin{align}
\dfrac{d\log(f(x))}{dx} = \dfrac{1}{f(x)}\dfrac{df(x)}{dx}
\end{align}

Therefore, 

\begin{align}
\dfrac{df(x)}{dx} = f(x)\dfrac{d\log(f(x))}{dx}
\end{align}

Let $f(\theta) = \Pi_{t=1}^T L_{nst}(\theta)$, and use the equation above, we can get the first order condition:

\begin{align}
0 = \Sigma_n \Sigma_s \dfrac{\pi_s\Pi_{t=1}^T L_{nst}(\theta)}{sum} \left(\dfrac{\Sigma_{t'}\partial \log L_{nst'}}{\partial\theta}\right)
\end{align}

where $sum$ is $\Sigma_{s'=1}^S \pi_{s'} \Pi_{t=1}^T L_{ns't}(\theta)$